# Сегментация текста

In [ ]:
!pip install pytesseract

In [ ]:
# текстовая основова:
# Обнаружение и извлечение текста из изображения с помощью Python _ by Roman _ NOP__Nuances of Programming _ Medium# описание: решается задача по сегментации картинки
# используется библиотека OpenCV
# чек-поинт 1
# цель - обработка/распознавание/сегментация
# результат = релизована автоматическая выгрузка файлов предварительно обработанных файлов:
# 1. rectanglebox.jng - сегментированный файл
# 2. .txt - текстовое содержание сегментов
# 3. threshold_image.jng - результат перевода в бинароное изображение
# 4. dilation_image.jng - результат применение метода расширения двоичного


import cv2
from pytesseract import pytesseract

path_to_tesseract = r"C:\Program Files\Tesseract-OCR\tesseract.exe"
pytesseract.tesseract_cmd = path_to_tesseract

img = cv2.imread("X51005230605.jpg")

# Преобразование "цветного" изображения в полутоновое для лучшей обработки текста
gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

# преобразование полутонового изображения в бинарное, чтобы повысить вероятность извлечения текста
ret, thresh1 = cv2.threshold(gray, 0, 255, cv2.THRESH_OTSU |
                                          cv2.THRESH_BINARY_INV)
cv2.imwrite('threshold_image.jpg',thresh1)
rect_kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (12, 12))

# применение метода расширения двоичного изображения для получения границ текста
dilation = cv2.dilate(thresh1, rect_kernel, iterations = 1)
cv2.imwrite('dilation_image.jpg',dilation)

contours, hierarchy = cv2.findContours(dilation, cv2.RETR_EXTERNAL,
                                            cv2.CHAIN_APPROX_NONE)

im2 = img.copy()

#формирование ограничительной рамки и сохранение файлов
for cnt in contours:
    x, y, w, h = cv2.boundingRect(cnt)
    rect = cv2.rectangle(im2, (x, y), (x + w, y + h), (0, 255, 0), 2)
    cropped = im2[y:y + h, x:x + w]
    cv2.imwrite('rectanglebox.jpg', rect)

for cnt in contours:
    x, y, w, h = cv2.boundingRect(cnt)
    rect = cv2.rectangle(im2, (x, y), (x + w, y + h), (0, 255, 0), 2)
    cropped = im2[y:y + h, x:x + w]
    cv2.imwrite('rectanglebox.jpg', rect)
    
    file = open("text_output2.txt", "a")
    text = pytesseract.image_to_string(cropped)

    file.write(text)
    file.write("\n")
    file.close

# Классификация именованных сущностей

In [1]:
# Импорт библиотек
import os
import glob
import json 
import random
from pathlib import Path
from difflib import SequenceMatcher


import cv2
import pandas as pd
import numpy as np
from PIL import Image
from tqdm import tqdm
from IPython.display import display
import matplotlib
from matplotlib import pyplot, patches

In [2]:
!wget https://www.dropbox.com/s/p51wpvtpf9to438/SROIE.zip?dl=1

--2023-01-20 10:47:17--  https://www.dropbox.com/s/p51wpvtpf9to438/SROIE.zip?dl=1
Resolving www.dropbox.com (www.dropbox.com)... 162.125.81.18, 2620:100:6020:18::a27d:4012
Connecting to www.dropbox.com (www.dropbox.com)|162.125.81.18|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: /s/dl/p51wpvtpf9to438/SROIE.zip [following]
--2023-01-20 10:47:17--  https://www.dropbox.com/s/dl/p51wpvtpf9to438/SROIE.zip
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://uc8c61fde13c2a6511d1124ea7a6.dl.dropboxusercontent.com/cd/0/get/B0508o2Bg2yl_EP9jDH9cTywpB4lLwADf9llkjR41IhP6MkKqJO-hjDCwoTmobUjAwDeIaGwalj0ZKqQlc9GhfAXHVHE-I4jYnoUdXuzo07y5lEwcQpsNqDFwU5yJCRfjQLa6r8y9LUNC1DND823ucalIXLJXLvPW1RIHwZP0Hksag/file?dl=1# [following]
--2023-01-20 10:47:17--  https://uc8c61fde13c2a6511d1124ea7a6.dl.dropboxusercontent.com/cd/0/get/B0508o2Bg2yl_EP9jDH9cTywpB4lLwADf9llkjR41IhP6MkKqJO-hjDCwoTmobUjAwDeIaGwalj0ZKqQlc9G

In [3]:
!unzip 'SROIE.zip?dl=1'

Archive:  SROIE.zip?dl=1
  inflating: SROIE2019/layoutlm-base-uncased/config.json  
  inflating: SROIE2019/layoutlm-base-uncased/pytorch_model.bin  
  inflating: SROIE2019/layoutlm-base-uncased/special_tokens_map.json  
  inflating: SROIE2019/layoutlm-base-uncased/tokenizer_config.json  
  inflating: SROIE2019/layoutlm-base-uncased/training_args.bin  
  inflating: SROIE2019/layoutlm-base-uncased/vocab.txt  
  inflating: SROIE2019/test/box/X00016469670.txt  
  inflating: SROIE2019/test/box/X00016469671.txt  
  inflating: SROIE2019/test/box/X51005200931.txt  
  inflating: SROIE2019/test/box/X51005230605.txt  
  inflating: SROIE2019/test/box/X51005230616.txt  
  inflating: SROIE2019/test/box/X51005230621.txt  
  inflating: SROIE2019/test/box/X51005230648.txt  
  inflating: SROIE2019/test/box/X51005230657.txt  
  inflating: SROIE2019/test/box/X51005230659.txt  
  inflating: SROIE2019/test/box/X51005268275.txt  
  inflating: SROIE2019/test/box/X51005268408.txt  
  inflating: SROIE2019/test/

In [4]:
!ls -l

total 854160
drwxr-xr-x 1 root root      4096 Jan 18 14:55  sample_data
drwxr-xr-x 5 root root      4096 Jan 20 10:48  SROIE2019
-rw-r--r-- 1 root root 874643738 Jan 20 10:48 'SROIE.zip?dl=1'


https://www.dropbox.com/s/p51wpvtpf9to438/SROIE.zip?dl=1 

In [5]:
# Загрузка файла
sroie_folder_path = Path('/content/SROIE2019')
example_file = Path('X51005577191.txt')

In [6]:
def read_bbox_and_words(path: Path):
  bbox_and_words_list = []

  with open(path, 'r', errors='ignore') as f:
    for line in f.read().splitlines():
      if len(line) == 0:
        continue
        
      split_lines = line.split(",")

      bbox = np.array(split_lines[0:8], dtype=np.int32)
      text = ",".join(split_lines[8:])

# Из разделенной строки сохраняем (имя файла, [точки ограничивающей рамки], текстовая строка).
# Имя файла пригодится в будущем
      bbox_and_words_list.append([path.stem, *bbox, text])
    
  dataframe = pd.DataFrame(bbox_and_words_list, columns=['filename', 'x0', 'y0', 'x1', 'y1', 'x2', 'y2', 'x3', 'y3', 'line'], dtype=np.int16)
  dataframe = dataframe.drop(columns=['x1', 'y1', 'x3', 'y3'])

  return dataframe

bbox_file_path = sroie_folder_path / "test/box" / example_file
print("== File content ==")
!head -n 5 "{bbox_file_path}"

bbox = read_bbox_and_words(path=bbox_file_path)
print("\n== Dataframe ==")
bbox.head(5)

== File content ==
83,246,648,246,648,277,83,277,FY EAGLE ENTERPRISE
282,277,447,277,447,304,282,304,002065464-X
185,307,542,307,542,337,185,337,NO. 42, JALAN PBS 14/11,
113,337,617,337,617,371,113,371,TAMAN PERINDUSTRIAN BUKIT SERDANG,
208,365,526,365,526,398,208,398,43300 SERI KEMBANGAN,

== Dataframe ==


<ipython-input-6-d754be8a72fd>:27: FutureWarning: Could not cast to int16, falling back to object. This behavior is deprecated. In a future version, when a dtype is passed to 'DataFrame', either all columns will be cast to that dtype, or a TypeError will be raised
  bbox = read_bbox_and_words(path=bbox_file_path)


,filename,x0,y0,x2,y2,line
0,X51005577191,83,246,648,277,FY EAGLE ENTERPRISE
1,X51005577191,282,277,447,304,002065464-X
2,X51005577191,185,307,542,337,"NO. 42, JALAN PBS 14/11,"
3,X51005577191,113,337,617,371,"TAMAN PERINDUSTRIAN BUKIT SERDANG,"
4,X51005577191,208,365,526,398,"43300 SERI KEMBANGAN,"


In [7]:
# Считаем файл сущностей, чтобы узнать, что маркировать в нашем тексте
def read_entities(path: Path):
  with open(path, 'r') as f:
    data = json.load(f)

  dataframe = pd.DataFrame([data])
  return dataframe

# Что получилось
entities_file_path = sroie_folder_path /  "test/entities" / example_file
print("== File content ==")
!head "{entities_file_path}"

entities = read_entities(path=entities_file_path)
print("\n\n== Dataframe ==")
entities



== File content ==
{
    "company": "FY EAGLE ENTERPRISE",
    "date": "07/12/2017",
    "address": "NO. 42, JALAN PBS 14/11, TAMAN PERINDUSTRIAN BUKIT SERDANG, 43300 SERI KEMBANGAN, SELANGOR",
    "total": "10.00"
}

== Dataframe ==


,company,date,address,total
0,FY EAGLE ENTERPRISE,07/12/2017,"NO. 42, JALAN PBS 14/11, TAMAN PERINDUSTRIAN B...",10.00


In [8]:
# Присвоить метку строке, проверив сходство строки и всех сущностей
def assign_line_label(line: str, entities: pd.DataFrame):
    line_set = line.replace(",", "").strip().split()
    for i, column in enumerate(entities):
        entity_values = entities.iloc[0, i].replace(",", "").strip()
        entity_set = entity_values.split()
        
        
        matches_count = 0
        for l in line_set:
            if any(SequenceMatcher(a=l, b=b).ratio() > 0.8 for b in entity_set):
                matches_count += 1
            
            if (column.upper() == 'ADDRESS' and (matches_count / len(line_set)) >= 0.5) or \
               (column.upper() != 'ADDRESS' and (matches_count == len(line_set))) or \
               matches_count == len(entity_set):
                return column.upper()

    return "O"


line = bbox.loc[1,"line"]
label = assign_line_label(line, entities)
print("Line:", line)
print("Assigned label:", label)

Line: 002065464-X
Assigned label: O


In [9]:
# В квитанции может быть только один товар, и его цена может совпадать с итоговой суммой, 
# поэтому дублируются этикетки. А может часть адреса тоже присутствует в конце квитанции.
# Чтобы игнорировать такие примеры, я написал простые жестко закодированные правила, 
# чтобы назначать итог и дату только самым большим ограничивающим прямоугольникам, 
# которые он мог найти (в зависимости от его площади), и не позволять назначать адрес после даты или итога.

def assign_labels(words: pd.DataFrame, entities: pd.DataFrame):
    max_area = {"TOTAL": (0, -1), "DATE": (0, -1)}  # Value, index
    already_labeled = {"TOTAL": False,
                       "DATE": False,
                       "ADDRESS": False,
                       "COMPANY": False,
                       "O": False
    }

# Проходимся по каждой строке и назначаем ей метку
    labels = []
    for i, line in enumerate(words['line']):
        label = assign_line_label(line, entities)

        already_labeled[label] = True
        if (label == "ADDRESS" and already_labeled["TOTAL"]) or \
           (label == "COMPANY" and (already_labeled["DATE"] or already_labeled["TOTAL"])):
            label = "O"

# Назначаем наибольшую ограничивающую рамку
        if label in ["TOTAL", "DATE"]:
            x0_loc = words.columns.get_loc("x0")
            bbox = words.iloc[i, x0_loc:x0_loc+4].to_list()
            area = (bbox[2] - bbox[0]) + (bbox[3] - bbox[1])

            if max_area[label][0] < area:
                max_area[label] = (area, i)

            label = "O"

        labels.append(label)

    labels[max_area["DATE"][1]] = "DATE"
    labels[max_area["TOTAL"][1]] = "TOTAL"

    words["label"] = labels
    return words


# Что получилось
bbox_labeled = assign_labels(bbox, entities)
bbox_labeled.head(15)

,filename,x0,y0,x2,y2,line,label
0,X51005577191,83,246,648,277,FY EAGLE ENTERPRISE,COMPANY
1,X51005577191,282,277,447,304,002065464-X,O
2,X51005577191,185,307,542,337,"NO. 42, JALAN PBS 14/11,",ADDRESS
3,X51005577191,113,337,617,371,"TAMAN PERINDUSTRIAN BUKIT SERDANG,",ADDRESS
4,X51005577191,208,365,526,398,"43300 SERI KEMBANGAN,",ADDRESS
5,X51005577191,305,398,424,426,SELANGOR,ADDRESS
6,X51005577191,117,429,614,459,TEL: 03-89433143 FAX: 03-89433143,O
7,X51005577191,214,460,513,486,GST NO: 001663156224,O
8,X51005577191,62,548,230,576,TAX INVOICE,O
9,X51005577191,60,606,636,635,RECEIPT #: CS00031383 DATE: 07/12/2017,DATE


In [10]:
# Разделение ограничивающих рамок на основе длины слова, вероятно, не лучший подход, но он достаточно хорош.

def split_line(line: pd.Series):
  line_copy = line.copy()

  line_str = line_copy.loc["line"]
  words = line_str.split(" ")

  # Filter unwanted tokens
  words = [word for word in words if len(word) >= 1]

  x0, y0, x2, y2 = line_copy.loc[['x0', 'y0', 'x2', 'y2']]
  bbox_width = x2 - x0
  

  new_lines = []
  for index, word in enumerate(words):
    x2 = x0 + int(bbox_width * len(word)/len(line_str))
    line_copy.at['x0', 'x2', 'line'] = [x0, x2, word]
    new_lines.append(line_copy.to_list())
    x0 = x2 + 5 

  return new_lines


# Example usage
new_lines = split_line(bbox_labeled.loc[1])
print("Original row:")
display(bbox_labeled.loc[1:1,:])

print("Splitted row:")
pd.DataFrame(new_lines, columns=bbox_labeled.columns)

Original row:


,filename,x0,y0,x2,y2,line,label
1,X51005577191,282,277,447,304,002065464-X,O


Splitted row:


,filename,x0,y0,x2,y2,line,label
0,X51005577191,282,277,447,304,002065464-X,O


In [11]:
# Собираем все вместе 
# Мы определили все наши функции, теперь нам просто нужно использовать их для каждого файла
# и преобразовать набор данных в формат, который может анализировать скрипт/модель.

from time import perf_counter
def dataset_creator(folder: Path):
  bbox_folder = folder / 'box'
  entities_folder = folder / 'entities'
  img_folder = folder / 'img'

# Сортировать по имени файла, чтобы при сжатии их вместе
# мы не получали какой-то другой файл (на всякий случай)
  entities_files = sorted(entities_folder.glob("*.txt"))
  bbox_files = sorted(bbox_folder.glob("*.txt"))
  img_files = sorted(img_folder.glob("*.jpg"))

  data = []

  print("Reading dataset:")
  for bbox_file, entities_file, img_file in tqdm(zip(bbox_files, entities_files, img_files), total=len(bbox_files)):            
    
# Чтение файла 
    bbox = read_bbox_and_words(bbox_file)
    entities = read_entities(entities_file)
    image = Image.open(img_file)

# Назначим метки строкам в bbox с помощью сущностей
    bbox_labeled = assign_labels(bbox, entities)
    del bbox

# Разделим строки на отдельные токены (части?)
    new_bbox_l = []
    for index, row in bbox_labeled.iterrows():
      new_bbox_l += split_line(row)
    new_bbox = pd.DataFrame(new_bbox_l, columns=bbox_labeled.columns, dtype=np.int16)
    del bbox_labeled


# Выполним более точное назначение меток, чтобы маркировка была более точной
    for index, row in new_bbox.iterrows():
      label = row['label']

      if label != "O":
        entity_values = entities.iloc[0, entities.columns.get_loc(label.lower())]
        entity_set = entity_values.split()
        
        if any(SequenceMatcher(a=row['line'], b=b).ratio() > 0.7 for b in entity_set):
            label = "S-" + label
        else:
            label = "O"
      
      new_bbox.at[index, 'label'] = label

    width, height = image.size
  
    data.append([new_bbox, width, height])

  return data
  

In [12]:
# Загрузка контрольной и тестовой выборки, для сохранения в переменную
dataset_train = dataset_creator(sroie_folder_path / 'train')
dataset_test = dataset_creator(sroie_folder_path / 'test')

Reading dataset:


  0%|          | 0/626 [00:00<?, ?it/s]<ipython-input-11-6847ce4fada3>:23: FutureWarning: Could not cast to int16, falling back to object. This behavior is deprecated. In a future version, when a dtype is passed to 'DataFrame', either all columns will be cast to that dtype, or a TypeError will be raised
  bbox = read_bbox_and_words(bbox_file)
<ipython-input-12-595c15ac69c5>:2: FutureWarning: Could not cast to int16, falling back to object. This behavior is deprecated. In a future version, when a dtype is passed to 'DataFrame', either all columns will be cast to that dtype, or a TypeError will be raised
  dataset_train = dataset_creator(sroie_folder_path / 'train')
100%|██████████| 626/626 [01:06<00:00,  9.46it/s]


Reading dataset:


  0%|          | 0/347 [00:00<?, ?it/s]<ipython-input-12-595c15ac69c5>:3: FutureWarning: Could not cast to int16, falling back to object. This behavior is deprecated. In a future version, when a dtype is passed to 'DataFrame', either all columns will be cast to that dtype, or a TypeError will be raised
  dataset_test = dataset_creator(sroie_folder_path / 'test')
100%|██████████| 347/347 [00:35<00:00,  9.65it/s]


In [13]:
#нормализация точек ограничивающих рамок

def normalize(points: list, width: int, height: int) -> list:
  x0, y0, x2, y2 = [int(p) for p in points]
  
  x0 = int(1000 * (x0 / width))
  x2 = int(1000 * (x2 / width))
  y0 = int(1000 * (y0 / height))
  y2 = int(1000 * (y2 / height))

  return [x0, y0, x2, y2]


def write_dataset(dataset: list, output_dir: Path, name: str):
  print(f"Writing {name}ing dataset:")
  with open(output_dir / f"{name}.txt", "w+", encoding="utf8") as file, \
       open(output_dir / f"{name}_box.txt", "w+", encoding="utf8") as file_bbox, \
       open(output_dir / f"{name}_image.txt", "w+", encoding="utf8") as file_image:

# Проходимся по каждому набору данных
      for datas in tqdm(dataset, total=len(dataset)):
        data, width, height = datas
        
        filename = data.iloc[0, data.columns.get_loc('filename')]

# Проходимся через каждую строку в наборе данных
        for index, row in data.iterrows():
          bbox = [int(p) for p in row[['x0', 'y0', 'x2', 'y2']]]
          normalized_bbox = normalize(bbox, width, height)

          file.write("{}\t{}\n".format(row['line'], row['label']))
          file_bbox.write("{}\t{} {} {} {}\n".format(row['line'], *normalized_bbox))
          file_image.write("{}\t{} {} {} {}\t{} {}\t{}\n".format(row['line'], *bbox, width, height, filename))

# Пишем новую строку, чтобы отделить данные друг от друга
        file.write("\n")
        file_bbox.write("\n")
        file_image.write("\n")

In [14]:
dataset_directory = Path('/content/output','dataset')

dataset_directory.mkdir(parents=True, exist_ok=True)

write_dataset(dataset_train, dataset_directory, 'train')
write_dataset(dataset_test, dataset_directory, 'test')

# Создание файла labels.txt для модели, какие категории предсказывать.
labels = ['COMPANY', 'DATE', 'ADDRESS', 'TOTAL']
IOB_tags = ['S']
with open(dataset_directory / 'labels.txt', 'w') as f:
  for tag in IOB_tags:
    for label in labels:
      f.write(f"{tag}-{label}\n")

# Пишет в последней метке O - предназначено для всех немаркированных слов
  f.write("O")

Writing training dataset:


100%|██████████| 626/626 [00:29<00:00, 21.16it/s]


Writing testing dataset:


100%|██████████| 347/347 [00:15<00:00, 22.21it/s]


In [15]:
%%bash
git clone https://github.com/microsoft/unilm.git
cd unilm/layoutlm/deprecated
pip install .

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Processing /content/unilm/layoutlm/deprecated
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 635.7/635.7 KB 41.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 195.7/195.7 KB 26.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 79.8 MB/s eta 0:00:00
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 93.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.5/7.5 MB 109.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 85.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 880.6/880.6 KB 78.2 MB/s eta 0:00:00
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with sta

Cloning into 'unilm'...


In [16]:
pretrained_model_folder_input= sroie_folder_path / Path('layoutlm-base-uncased') # Define it so we can copy it into our working directory

pretrained_model_folder=Path('/content/output/layoutlm-base-uncased/') 
label_file=Path(dataset_directory, "labels.txt")

# Move to the script directory
os.chdir("/content/unilm/layoutlm/deprecated/examples/seq_labeling")


In [17]:
! cp -r "{pretrained_model_folder_input}" "{pretrained_model_folder}"
! sed -i 's/"num_attention_heads": 16,/"num_attention_heads": 12,/' "{pretrained_model_folder}/"config.json

In [18]:
! cat "/content/SROIE2019/layoutlm-base-uncased/config.json"

{
  "attention_probs_dropout_prob": 0.1,
  "finetuning_task": null,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "is_decoder": false,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "max_2d_position_embeddings": 1024,
  "num_attention_heads": 16,
  "num_hidden_layers": 12,
  "num_labels": 2,
  "output_attentions": false,
  "output_hidden_states": false,
  "output_past": true,
  "pruned_heads": {},
  "torchscript": false,
  "type_vocab_size": 2,
  "use_bfloat16": false,
  "vocab_size": 30522
}

In [19]:
! rm -rf /content/output/dataset/cached*

In [20]:
! python run_seq_labeling.py \
                            --data_dir /content/output/dataset \
                            --labels /content/output/dataset/labels.txt \
                            --model_name_or_path "{pretrained_model_folder}" \
                            --model_type layoutlm \
                            --max_seq_length 512 \
                            --do_lower_case \
                            --do_train \
                            --num_train_epochs 10 \
                            --logging_steps 50 \
                            --save_steps -1 \
                            --output_dir output \
                            --overwrite_output_dir \
                            --per_gpu_train_batch_size 8 \
                            --per_gpu_eval_batch_size 16

Epoch:   0% 0/10 [00:00<?, ?it/s]
Iteration:   0% 0/79 [00:00<?, ?it/s]/usr/local/lib/python3.8/dist-packages/transformers/optimization.py:155: UserWarning: This overload of add_ is deprecated:
	add_(Number alpha, Tensor other)
Consider using one of the following signatures instead:
	add_(Tensor other, *, Number alpha) (Triggered internally at ../torch/csrc/utils/python_arg_parser.cpp:1420.)
  exp_avg.mul_(beta1).add_(1.0 - beta1, grad)

Iteration:   1% 1/79 [00:03<04:18,  3.31s/it]
Iteration:   3% 2/79 [00:04<02:17,  1.79s/it]
Iteration:   4% 3/79 [00:04<01:39,  1.30s/it]
Iteration:   5% 4/79 [00:05<01:20,  1.08s/it]
Iteration:   6% 5/79 [00:06<01:11,  1.04it/s]
Iteration:   8% 6/79 [00:06<01:04,  1.13it/s]
Iteration:   9% 7/79 [00:07<01:00,  1.20it/s]
Iteration:  10% 8/79 [00:08<00:57,  1.24it/s]
Iteration:  11% 9/79 [00:09<00:54,  1.28it/s]
Iteration:  13% 10/79 [00:09<00:52,  1.30it/s]
Iteration:  14% 11/79 [00:10<00:51,  1.32it/s]
Iteration:  15% 12/79 [00:11<00:50,  1.32it/s]
Ite

In [21]:
# Evaluate for test set and make predictions
! python run_seq_labeling.py \
                            --data_dir /content/output/dataset \
                            --labels /content/output/dataset/labels.txt \
                            --model_name_or_path "{pretrained_model_folder}" \
                            --model_type layoutlm \
                            --do_lower_case \
                            --max_seq_length 512 \
                            --do_predict \
                            --logging_steps 10 \
                            --save_steps -1 \
                            --output_dir output \
                            --per_gpu_eval_batch_size 8

Evaluating: 100% 44/44 [00:11<00:00,  3.85it/s]


In [22]:
cat output/test_results.txt

f1 = 0.9563007296876281
loss = 0.07875144496111633
precision = 0.9456786119669207
recall = 0.9671641791044776


In [23]:
import cv2
from matplotlib import pyplot, patches
import matplotlib

data = pd.read_csv('/content/output/dataset/test_image.txt', delimiter="\t", names=["name", "bbox", "size", "image"])
data_category = pd.read_csv('/content/output/dataset/test.txt', delimiter="\t", names=["name", "true_category"]).drop(columns=["name"])
data_prediction_category = pd.read_csv("output/test_predictions.txt", delimiter=" ", names=["name", "prediction_category"]).drop(columns=["name"])

data_merge = data.merge(data_category, left_index=True, right_index=True)
merged = data_merge.merge(data_prediction_category, left_index=True, right_index=True)
merged_groups = list(merged.groupby("image"))